In [1]:
import os
import pandas as pd

import dask.dataframe as dd

In [2]:
DATA_DIR = '../../data/raw'
LOOKUPS = DATA_DIR + "/Lookups"
TRANSACT = DATA_DIR + "/Transactions"

In [3]:
files = os.listdir(TRANSACT)

In [9]:
stores = pd.read_csv(LOOKUPS + '/stores.txt', sep='\t')
drugs = pd.read_csv(LOOKUPS + '/Drug_LookUp.txt', sep='\t')
illness = pd.read_csv(LOOKUPS + '/ChronicIllness_LookUp.txt', sep='\t')
patients = pd.read_csv(LOOKUPS + '/patients.txt', sep='\t')
atc = pd.read_csv(LOOKUPS + '/ATC_LookUp.txt', sep='\t')

In [10]:
t1 = illness.merge(drugs, on='MasterProductID', how='outer')

In [5]:
def get_subset():

    samples = []
    for file in files:
        df = pd.read_csv(TRANSACT + '/' + file, sep='\t')
        sample = df.sample(frac=.1)
        samples.append(sample)

    return pd.concat(samples)
    
    

In [6]:
subset = get_subset()

In [11]:
subset = subset.merge(t1, left_on='Drug_ID', right_on='MasterProductID', suffixes=('_illness', '_drug'), how='outer')

In [12]:
subset.head()

,Patient_ID,Store_ID,Prescriber_ID,Drug_ID,SourceSystem_Code,Prescription_Week,Dispense_Week,Drug_Code,NHS_Code,IsDeferredScript,...,ManufacturerCode,ManufacturerName,ManufacturerGroupID,ManufacturerGroupCode,ChemistListPrice,ATCLevel5Code,ATCLevel4Code,ATCLevel3Code,ATCLevel2Code,ATCLevel1Code
0,7716.0,2636.0,6951.0,6470.0,F,2012-09-09,2013-02-17,PANA8,8814X,0.0,...,GC,GLAXOSMITHKLINE AUSTRALIA,105,GLAXOSMITHKLINE CONSUMER,4.44,N02BE01,N02BE,N02B,N02,N
1,9513.0,1038.0,55907.0,6470.0,F,2011-07-17,2012-03-04,PANA8,8814X,0.0,...,GC,GLAXOSMITHKLINE AUSTRALIA,105,GLAXOSMITHKLINE CONSUMER,4.44,N02BE01,N02BE,N02B,N02,N
2,9407.0,2539.0,7279.0,6470.0,F,2011-11-06,2012-03-04,PANA8,8814X,0.0,...,GC,GLAXOSMITHKLINE AUSTRALIA,105,GLAXOSMITHKLINE CONSUMER,4.44,N02BE01,N02BE,N02B,N02,N
3,6534.0,2275.0,24225.0,6470.0,F,2014-05-04,2014-11-23,PANA8,8814X,0.0,...,GC,GLAXOSMITHKLINE AUSTRALIA,105,GLAXOSMITHKLINE CONSUMER,4.44,N02BE01,N02BE,N02B,N02,N
4,6238.0,994.0,59754.0,6470.0,F,2015-07-26,2015-12-20,PANA8,8814X,0.0,...,GC,GLAXOSMITHKLINE AUSTRALIA,105,GLAXOSMITHKLINE CONSUMER,4.44,N02BE01,N02BE,N02B,N02,N
